In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('/kaggle/input/playground-series-s3e12/train.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

1. There is no null value.

In [ ]:
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

In [ ]:
px.histogram(train_df , x = 'target')

In [ ]:
px.histogram(train_df , x = 'gravity',marginal="box")

1. Lesser the gravity, lesser the chance of getting stones. 


In [ ]:
px.histogram(train_df , x = 'ph',marginal="box")

Low ph cause kidney stones. 

In [ ]:
px.histogram(train_df , x = 'cond',marginal="box")

Majorly conductivity range of 20mMho - 30mMho cause kidney stones. 

In [ ]:
px.histogram(train_df , x = 'calc',marginal="box")

Larger the amount of calcium present in the urine, larger the chance of kidney stones.

In [ ]:
train_df.corr()

In [ ]:
sns.heatmap(train_df.corr(),annot=True)

This heatmap proves our previous observation.

#### Modeling

In [ ]:
import xgboost as xgb
from sklearn import metrics

In [ ]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain['target'].values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics='auc', early_stopping_rounds=early_stopping_rounds, verbose_eval=True)
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['target'],eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])
    dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
        
    #Print model report:
    print("\nModel Report")
    print("Accuracy : %.4g" % metrics.accuracy_score(dtrain['target'].values, dtrain_predictions))
    print("AUC Score (Train): %f" % metrics.roc_auc_score(dtrain['target'], dtrain_predprob))


In [ ]:
predictors = [x for x in train_df.columns if x not in ['target', 'id']]
xgb1 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=500,
 max_depth=3,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 seed=27)
modelfit(xgb1, train_df, predictors)

In [ ]:
from sklearn.model_selection import GridSearchCV

param_test1 = {
 'max_depth':range(1,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=17, max_depth=3,
 min_child_weight=1, objective= 'binary:logistic',seed=27), 
param_grid = param_test1, scoring='roc_auc',n_jobs=4, cv=5)
gsearch1.fit(train_df[predictors],train_df['target'])
gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.1, n_estimators=17, max_depth=1,
 min_child_weight=1, gamma=0, objective= 'binary:logistic',seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4, cv=5)
gsearch3.fit(train_df[predictors],train_df['target'])
gsearch3.best_params_, gsearch3.best_score_

In [ ]:
predictors = [x for x in train_df.columns if x not in ['target', 'id']]
xgb1 = xgb.XGBClassifier(
 learning_rate =0.1,
 n_estimators=500,
 max_depth=3,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 seed=27)
modelfit(xgb1, train_df, predictors)

In [ ]:
param_test1 = {
 'max_depth':range(1,10,2),
 'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.01, n_estimators=29, max_depth=3,
 min_child_weight=1, objective= 'binary:logistic',seed=27), 
param_grid = param_test1, scoring='roc_auc',n_jobs=4, cv=5)
gsearch1.fit(train_df[predictors],train_df['target'])
gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = xgb.XGBClassifier( learning_rate =0.01, n_estimators=29, max_depth=3,
 min_child_weight=1, gamma=0, objective= 'binary:logistic',seed=27), 
 param_grid = param_test3, scoring='roc_auc',n_jobs=4, cv=5)
gsearch3.fit(train_df[predictors],train_df['target'])
gsearch3.best_params_, gsearch3.best_score_

In [ ]:
from sklearn.model_selection import train_test_split

X = train_df.drop(['target'], axis =1)
y = train_df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
xgb_classifier1 = xgb.XGBClassifier( learning_rate =0.1,
 n_estimators=17,
 max_depth=1,min_child_weight=1,)
xgb_classifier1.fit(X_train,y_train)
xgb_classifier1.score(X_train, y_train)

In [ ]:
xgb_classifier1.score(X_test, y_test)

#### Test Prediction

In [ ]:
test_df = pd.read_csv('/kaggle/input/playground-series-s3e12/test.csv')
Y_pred = xgb_classifier1.predict_proba(test_df)[:,1]

In [ ]:
submission = pd.DataFrame({
        "id": test_df["id"],
        "target": Y_pred
    })
submission.to_csv('submission.csv', index=False)